In [ ]:
from astropy.table import Table, join
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

# Load the data
train_provabgs = Table.read(
    "/mnt/ceph/users/polymathic/astroclip/datasets/provabgs/provabgs_paired_train_embeddings.hdf5"
)
test_provabgs = Table.read(
    "/mnt/ceph/users/polymathic/astroclip/datasets/provabgs/provabgs_paired_test_embeddings.hdf5"
)

In [ ]:
models = ["astrodino"]

# Get data
data = {}
for model in models:
    data[model] = {}
    X_train, X_test = (
        train_provabgs[model + "_embeddings"],
        test_provabgs[model + "_embeddings"],
    )
    embedding_scaler = StandardScaler().fit(X_train)
    data[model]["train"] = embedding_scaler.transform(X_train)
    data[model]["test"] = embedding_scaler.transform(X_test)

# Get redshifts
train_redshift = train_provabgs["Z_HP"].reshape(-1, 1)
test_redshift = test_provabgs["Z_HP"].reshape(-1, 1)

# Scale redshifts
scaler = StandardScaler().fit(train_redshift)
train_redshift = scaler.transform(train_redshift)

In [ ]:
def zero_shot_train(X_train, y_train, X_test):
    neigh = KNeighborsRegressor(weights="distance", n_neighbors=64)
    neigh.fit(X_train, y_train)
    preds = neigh.predict(X_test)
    return preds

def few_shot_train(X_train, y_train, X_test)

In [ ]:
preds, truths = {}, {}

for model in data.keys():
    raw_preds = zero_shot_train(
        data[model]["train"], train_redshift, data[model]["test"]
    )
    preds[model] = scaler.inverse_transform(raw_preds)
    truths[model] = test_redshift

In [ ]:
r2_score(truths["astrodino"], preds["astrodino"])